In [1]:
#import libraries
import nltk
import numpy as np
import random
import string # to process standard python strings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import re
import pickle
import flask

In [2]:
#open previous lines text
f=open('previous.txt','r',errors = 'ignore')

In [3]:
raw=f.read()

In [4]:
# initializing bad_chars_list 
bad_chars = ['\n'] 
for i in bad_chars : 
    raw = raw.replace(i, ' ') 

In [5]:
# initializing bad_chars_list number 2 
bad_chars = ['\''] 
for i in bad_chars : 
    raw = raw.replace(i, "'") 

In [6]:
raw=raw.lower()# converts to lowercase

In [7]:
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to /home/jone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

In [9]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.

In [10]:
#lemmatization functions
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [11]:
#if user inputs standard greetings the chatbot will output the following
GREETING_INPUTS = ("hello", "hi", "greetings", "how's it going?", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [12]:
#open convo.pickle dictionary with {previous line:Cartman's line that follows}
with open('convo.pickle', 'rb') as handle:
    convo_dict = pickle.load(handle)

In [13]:
#this function returns Cartman's lines that follow the previous line 
#with the least cosine distance from the user input
#if the user input line is not similar to any of the previous line
#chatbot will output that it does not understand
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2))
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf == 0):
        robo_response=robo_response+"AWESOM-O does not understand"
        return robo_response
    else:
        for i in convo_dict:
            if sent_tokens[idx] in i:
                response = convo_dict[i]
            else:
                continue
        robo_response = robo_response+response 
        return robo_response

In [14]:
#additional cleaning function
def replace_value_with_definition(key_to_find, definition):
    for key in convo_dict.keys():
        if key == key_to_find:
            convo_dict[key] = definition

replace_value_with_definition('what do you think happens if you do have it?',            
                              'they drag you out of here, put you in this big containment facility, where they stick a cold metal pipe up your *** with this clear jelly, and then they shock your *****. ')

In [ ]:
flag=True
print('AWESOM-O: Greetings. I am the AWESOM-O 4000. AWESOM-O is here to answer your questions. Type "shut down" to end conversation')
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='shut down'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("AWESOM-O: You are welcome")
        else:
            if(greeting(user_response)!=None):
                print("AWESOM-O: "+greeting(user_response))
            else:
                print("AWESOM_O: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("AWESOM-O: AWESOM-O will now initiate shut down sequence. Goodbye!")

AWESOM-O: Greetings. I am the AWESOM-O 4000. AWESOM-O is here to answer your questions. Type "shut down" to end conversation
hi
AWESOM-O: *nods*
are you okay awesom-o?
AWESOM_O: 

/home/jone/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


yes. awesom-o is fine. please go on.
are you still pretending to be a robot?
AWESOM_O: shh!  i just need to uh make butters think i'm a robot for a little while longer.
how much wood would a woodchuck chuck if a woodchuck could chuck wood?
AWESOM_O: seventeen. 
do you play video games?
AWESOM_O: the last two weeks we've been too busy to play video games and, look at what we did. there's been drama, action, romance... i mean honestly you guys, do we need video games to play?
can you come up with an idea for a movie that will make more than a hundred million box office?
AWESOM_O: um... okay. how about this:  adam sandler is like, in love with some girl, but then it turns out that the girl is actually a... golden retriever, or something.
who is your best friend?
AWESOM_O: you are, jimmy! we've always been best friends. we know everything about each other.
what are you talking about?
AWESOM_O: really?
forget what?
AWESOM_O: eh eh, you're right, colonel sanders! you shouldn't give her any m

In [15]:
#flask app application 
# Initialize the app
app = flask.Flask(__name__)

@app.route("/")
def viz_page():
    with open("chatbot.html", 'r') as viz_file:
        return viz_file.read()

@app.route("/gof", methods=["POST"])
def chatbot():
    """
    When A POST request with json data is made to this url,
    Read the grid from the json, update and send it back
    """
    flag=1
    data = flask.request.json
    a = data['text']
    print(a)
    user_response=a.lower()
    if(user_response!='shut down'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=0
            b = "AWESOM-O: You are welcome"
        else:
            if(greeting(user_response)!=None):
                b = "AWESOM-O: "+greeting(user_response)
            else:
                b = "AWESOM_O: " +  response(user_response)
                #b = response(user_response)
                sent_tokens.remove(user_response)
    else:
        flag=0
        b = "AWESOM-O: AWESOM-O will now initiate shut down sequence. Goodbye!"
    
    print(b)
    print (flask.jsonify({'text': b, 'flag': flag}))
    return flask.jsonify({'text': b, 'flag': flag})

In [ ]:

#--------- RUN WEB APP SERVER ------------#

# Start the app server on port 80
# (The default website port)
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Mar/2019 17:28:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2019 17:28:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2019 17:28:52] "POST /gof HTTP/1.1" 200 -


hey
AWESOM-O: hello
<Response 36 bytes [200 OK]>
sup


/home/jone/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


cartman sucks


...

127.0.0.1 - - [21/Mar/2019 17:29:15] "POST /gof HTTP/1.1" 200 -


AWESOM_O: that's fine! i'd like playing with myself! i'll play with myself all day long!  what?
<Response 116 bytes [200 OK]>
fatass


127.0.0.1 - - [21/Mar/2019 17:30:15] "POST /gof HTTP/1.1" 200 -


AWESOM_O: yeah. i sure did. i don't think i should record that album now.
<Response 94 bytes [200 OK]>


127.0.0.1 - - [21/Mar/2019 17:31:33] "GET / HTTP/1.1" 200 -
